In [ ]:
#konlpy 설치
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 35.1 MB/s eta 0:00:00


In [ ]:
!pip install summa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 656.7 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for summa: filename=summa-1.2.0-py3-none-any.whl size=54388 sha256=8c31e1ddaf39bca50ebc988c8cf64ee3d043f42d27b182d3f3bec72ebaf5106e
  Stored in directory: /root/.cache/pip/wheels/4a/ca/c5/4958614cfba88ed6ceb7cb5a849f9f89f9ac49971616bc919f
Successfully built summa


In [ ]:
!pip install -U --no-cache-dir gdown --pre
!gdown --no-cookies --id 1KuBG40WNpVPV1ilfGaiCI2D3l8JVSfbS

  Attempting uninstall: gdown
    Found existing installation: gdown 4.6.6
    Uninstalling gdown-4.6.6:
      Successfully uninstalled gdown-4.6.6
/usr/local/lib/python3.10/dist-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (uriginal): https://drive.google.com/uc?id=1KuBG40WNpVPV1ilfGaiCI2D3l8JVSfbS
From (redirected): https://drive.google.com/uc?id=1KuBG40WNpVPV1ilfGaiCI2D3l8JVSfbS&confirm=t&uuid=532c058f-ea01-4369-be83-1767823ce87c
To: /content/train_original.json
100% 800M/800M [00:18<00:00, 43.6MB/s]


In [ ]:
from konlpy.tag import Hannanum, Okt, Kkma

In [265]:
from summa import keywords
from sklearn.metrics import jaccard_score
import gensim
from gensim.models.word2vec import Word2Vec
import re
import pandas as pd
from tqdm import tqdm, tqdm_pandas
import gensim
import pickle
tqdm_pandas(tqdm())
hannanum = Hannanum()
kkma = Kkma()
okt = Okt()

0it [00:00, ?it/s]
0it [00:36, ?it/s]


In [455]:
with open('ffinal_data.pkl', 'rb') as f :
    li = pickle.load(f)

In [106]:
stopwords = ['저자', '작가', '소개', '인기', '우리', '계기', '보유', '내용', '이야기']

In [128]:
def preprocess(string) :
    string = ' '.join(re.findall('[가-힣]+|[a-z]+', string.lower())) # 특수문자들 제거 및 모든 영어 소문자 변환
    string = okt.nouns(string)
    for ind, i in enumerate(string) :
        if i in stopwords :
            string[ind] = ''
    string = ' '.join(string)
    string = ' '.join(re.findall('\w{2,}', string.lower())) # 1글자인 단어들 삭제
    words_n = 5 # 뽑을 키워드 갯수

    if len(set(string.split())) < words_n :
        words_n = len(set(string.split())) # 단어 갯수가 적을 경우 뽑을 키워드 갯수를 낮춘다

    string = keywords.keywords(string, words = words_n, scores = True) # 키워드 뽑기

    word = []
    score = []
    for i, j in string :
        i = ''.join(i).split()
        word.append(i)
    word_list = []
    for i in word :
        for j in i :
            word_list.append(j)
    for i in word_list : # 단어들 1개씩
        for ind, j in enumerate(word) :
            if i in j :
                score.append([i, round(string[ind][1], 2)])
    if len(score) > 5 :
        score = score[:5]

    return score

In [ ]:
for ind, i in enumerate(a['소개글']) : # 테스트용 for문
    ['소개글'][ind] = preprocess(i)

In [ ]:
# li['제목 명사'] = li['상품명'].progress_apply(lambda a : preprocess(str(a))) # 전처리 함수 실행

In [129]:
li['소개글 키워드'] = li['소개글'].progress_apply(lambda a : preprocess(str(a)))

100%|██████████| 10165/10165 [17:21<00:00,  9.76it/s]


In [132]:
li['목차 키워드'] = li['목차'].progress_apply(lambda a : preprocess(str(a)))

100%|██████████| 10165/10165 [27:07<00:00,  6.25it/s]


In [163]:
li.to_pickle('keyword_final.pkl')

In [113]:
li[['목차 키워드', '소개글 키워드']].to_pickle('keyword.pkl')

In [456]:
li

,판매상품ID,상품명,상품명_Hannanum,상품명_okt,상품명_Kkma,상품명_Komoran,정가,판매가,저자,출판사,발행(출시)일자,분야,소개글,소개글_키워드,목차,목차_키워드
0,S000211563502,에디토리얼 씽킹,"[에디토리얼, 씽킹]","[에디, 토, 리얼, 킹]","[에디토, 에디토리얼, 리얼, 씽, 씽킹, 킹]","[에디, 토리, 얼]",20000,18000,최혜진,터틀넥프레스,2023-12-22,예술/대중문화,"하늘 아래 더 이상 새로울 것이 없는 시대,\n기획자ㆍ창작는 어떻게 ‘차이’를 만들...","{'에디터': 0.23, '창조': 0.22, '시대': 0.21, '기획': 0....",Prologue\n에디토리얼 씽킹을 시작합니다.\n1. 재료 수집 : 가능성을 품은...,"{'재료': 0.32, '거리': 0.27, '가능성': 0.22, '리얼': 0...."
1,S000202934939,창조적 행위: 존재의 방식,"[창조적, 행위, 존재, 방식]","[창조, 행위, 존재, 방식]","[창조적, 행위, 존재, 방식]","[창조, 행위, 존재, 방식]",19800,17820,릭 루빈,코쿤북스,2023-07-10,예술/대중문화,"『뉴욕 타임스』, 『선데이 타임스』 베스트셀러 1위\n2023년 1월 출간 후 미국...","{'루빈': 0.37, '우리': 0.21, '예술가': 0.21, '창조': 0....",78가지 생각 지대\n\n저자들의 말\n누구나 창조한다\n신호를 받는 안테나처럼\n...,"{'예술': 0.32, '단계': 0.23, '저자': 0.16, '시간': 0.1..."
2,S000001056702,방구석 미술관,"[방구석, 미술관]","[방, 구석, 미술관]","[방구석, 미술관]","[방구석, 미술관]",16800,15120,조원재,블랙피쉬,2021-12-21,예술/대중문화,세상에서 가장 술술 읽히는 미술책!\n《방구석 미술관》은 2018년 출간 이래 방송...,"{'화가': 0.44, '미술': 0.35, '구석': 0.35, '영혼': 0.1...","들어가며\n01. 죽음 앞에 절규한 에드바르트 뭉크, 사실은 평균 수명을 높인 장수...","{'사실': 0.4, '미술': 0.27, '화가': 0.2, '사랑': 0.2, ..."
3,S000211518386,홀리데이 인 뮤지엄,"[홀리데이, 뮤지엄]","[홀리데이, 인, 뮤, 지엄]","[홀, 홀리데이, 리, 데이, 뮤]","[홀리데이 인, 뮤지, 엄]",19000,17100,한이준,흐름출판,2023-12-04,예술/대중문화,"★\n10년간 70개 이상, 3,000회 이상의 해설을 진행한 도슨트에게 ‘직접’ ...","{'화가': 0.35, '미술관': 0.23, '소개': 0.23, '박수근': 0...","추천의 글\n프롤로그_ 내 작은 가방 속, 최소한의 미술 이야기\n\n1부. 국내 ...","{'둘러보기': 0.35, '미술관': 0.27, '화가': 0.25, '그림': ..."
4,S000211613105,포 어스 For Earth For Us,"[포, 어스]","[포, 어스]","[포, 어스]","[포, 어스]",35000,31500,케이채,웨일북(whalebooks),2023-12-11,예술/대중문화,"“내가 사랑하는 모든 색은 지구에게 배웠습니다”\n\n총 2,500일, 2만 킬로미...","{'지구': 0.35, '사진': 0.28, '우리': 0.21, '대자연': 0....",프롤로그_자연이야말로 가장 위대한 화가입니다\nMap of Color Travel\...,"{'남극': 0.22, '제도': 0.2, '국립': 0.2, '공원': 0.2, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10160,S000208801672,C언어 이해,"[언어, 이해]","[언어, 이해]","[언어, 이해]","[언어, 이해]",23000,23000,김기용 외,그린,2023-08-25,컴퓨터/IT,"김기용의 『C언어 이해』는 〈C언어의 배경과 기본 구성 요소〉, 〈변수와 연산자〉,...","{'언어': 0.48, '변수': 0.32, '연산자': 0.32, '조건문': 0...",제1장 C언어의 배경과 기본 구성 요소\n1.1 컴퓨터와 소프트웨어\n1.2 프로그...,"{'함수': 0.41, '연습': 0.29, '변수': 0.29, '포인터': 0...."
10161,S000208797452,로봇과 AI의 인류학,"[로봇, 의, 인류학]","[로봇, 의, 인류학]","[로봇, 인류학]","[로봇, 의, 인류학]",24000,21600,캐슬린 리처드슨,눌민,2023-08-18,인문,"로봇과 AI 그리고 인간,\n인류의 문화와 사회에 대해 근원적이고 본질적인 질문을 ...","{'로봇': 0.44, '학자': 0.36, '진짜': 0.36, '현장': 0.2...","감사의 말ㆍ7\n\n머리말: 절멸불안과 기계ㆍ11\n인류여, 너희의 종말을 경계하라...","{'기계': 0.43, '로봇': 0.42, '신체': 0.26, '사회': 0.1..."
10162,S000208797234,내일은 로봇왕 7: 충격의 인터넷 방송,"[내일, 로봇왕, 충격, 인터넷, 방송]","[내일, 로봇, 충격, 인터넷, 방송]","[내일, 로봇, 로봇왕, 왕, 충격, 인터넷, 방송]","[내일, 로봇, 왕, 충격, 인터넷 방송]",15500,13950,포도알친구,미래엔아이세움,2023-08-29,어린이(초등),로봇에 담긴 과학 원리를 이해하고\n나만의 로봇을 만들어 보자!\n로봇에 대한 모든...,"{'로봇': 0.45, '과학': 0.45, '대회': 0.14, '마음': 0.1...","제1화 더욱 치열해진 2차전 8\n로봇 POINT 다관절 로봇, 생체모방 로봇\n로...","{'로봇': 0.9, '사이배': 0.09, '슬론': 0.09, '소프트웨어': ..."
10163,S000208797129,딥러닝 기초,"[딥러닝, 기초]","[딥, 러닝, 기초]","[딥, 딥러닝, 러닝, 기초]",[기초],20000,19000,이강혁 외,구미서관,2023-09-13,기술/공학,앞으로 인공지능을 활용할 수 있는 기술자들은 그렇지 못한 기술자에 비하여 생산성이나...,"{'건설': 0.32, '기술': 0.32, '인공': 0.29, '지능': 0.2...",PART 1 Basics of Deep Learning\nCHAPTER 01 4차산...,"{'모델': 0.38, '신경망': 0.32, '분류기': 0.24, '개발': 0..."


###실행 파일

In [412]:
def JarccardSimilarity (input_key, data_key) :
    score = 0

    in_keys = set(input_key.keys())
    da_keys = set(data_key.keys())
    same = in_keys & da_keys
    for i in list(same) :
        score += (input_key[i] * 1.7) + (data_key[i] * 1.3)
    a = set.union(in_keys ,da_keys)
    return score / len(a)

In [461]:
def JarccardSimilarity_title (input_key, data_key) :
    score = 0

    in_keys = set(input_key.keys())
    da_keys = set(data_key)
    same = in_keys & da_keys
    for i in list(same) :
        score += (input_key[i] * 1.7) + 1.3
    a = set.union(in_keys, da_keys)
    return score / len(a)

In [462]:
intro_score = li['소개글_키워드'].apply(lambda a : JarccardSimilarity(test, a))
index_score = li['목차_키워드'].apply(lambda a : JarccardSimilarity(test, a))
title_score = li['상품명_Hannanum'].apply(lambda a : JarccardSimilarity_title(test, a))

In [469]:
intro_sim_list = intro_score.sort_values()[-1:-6:-1]
index_sim_list = index_score.sort_values()[-1:-6:-1]
title_sim_list = title_score.sort_values()[-1:-6:-1]

In [471]:
title_sim_list.keys()

Int64Index([2514, 1794, 5433, 5793, 2611], dtype='int64')

In [470]:
title_sim_list

2514    0.372200
1794    0.372200
5433    0.310167
5793    0.310167
2611    0.273333
Name: 상품명_Hannanum, dtype: float64

In [466]:
li.loc[2514]

판매상품ID                                              S000203179638
상품명                                                      차가운 자본주의
상품명_Hannanum                                               [자본주의]
상품명_okt                                                    [자본주의]
상품명_Kkma                                                   [자본주의]
상품명_Komoran                                                [자본주의]
정가                                                          17500
판매가                                                         15750
저자                                                           윤루카스
출판사                                                     떠오름(RISE)
발행(출시)일자                                      2023-07-21 00:00:00
분야                                                          경제/경영
소개글             차갑지만 아름다운 자본주의 시장경제\n\n반년 만에 40만 경제 유튜버가 된\n윤루...
소개글_키워드         {'저자': 0.46, '자본주의': 0.34, '경제': 0.23, '대한': 0...
목차              작가의 말_ 초판 절판시켜 버렸다\n프롤로그_ 당신을 위해 글을 쓰지 않는다\n\n...
목차_키워드    

In [467]:
li[li['분야'] == '예술/대중문화']

,판매상품ID,상품명,상품명_Hannanum,상품명_okt,상품명_Kkma,상품명_Komoran,정가,판매가,저자,출판사,발행(출시)일자,분야,소개글,소개글_키워드,목차,목차_키워드
0,S000211563502,에디토리얼 씽킹,"[에디토리얼, 씽킹]","[에디, 토, 리얼, 킹]","[에디토, 에디토리얼, 리얼, 씽, 씽킹, 킹]","[에디, 토리, 얼]",20000,18000,최혜진,터틀넥프레스,2023-12-22,예술/대중문화,"하늘 아래 더 이상 새로울 것이 없는 시대,\n기획자ㆍ창작는 어떻게 ‘차이’를 만들...","{'에디터': 0.23, '창조': 0.22, '시대': 0.21, '기획': 0....",Prologue\n에디토리얼 씽킹을 시작합니다.\n1. 재료 수집 : 가능성을 품은...,"{'재료': 0.32, '거리': 0.27, '가능성': 0.22, '리얼': 0...."
1,S000202934939,창조적 행위: 존재의 방식,"[창조적, 행위, 존재, 방식]","[창조, 행위, 존재, 방식]","[창조적, 행위, 존재, 방식]","[창조, 행위, 존재, 방식]",19800,17820,릭 루빈,코쿤북스,2023-07-10,예술/대중문화,"『뉴욕 타임스』, 『선데이 타임스』 베스트셀러 1위\n2023년 1월 출간 후 미국...","{'루빈': 0.37, '우리': 0.21, '예술가': 0.21, '창조': 0....",78가지 생각 지대\n\n저자들의 말\n누구나 창조한다\n신호를 받는 안테나처럼\n...,"{'예술': 0.32, '단계': 0.23, '저자': 0.16, '시간': 0.1..."
2,S000001056702,방구석 미술관,"[방구석, 미술관]","[방, 구석, 미술관]","[방구석, 미술관]","[방구석, 미술관]",16800,15120,조원재,블랙피쉬,2021-12-21,예술/대중문화,세상에서 가장 술술 읽히는 미술책!\n《방구석 미술관》은 2018년 출간 이래 방송...,"{'화가': 0.44, '미술': 0.35, '구석': 0.35, '영혼': 0.1...","들어가며\n01. 죽음 앞에 절규한 에드바르트 뭉크, 사실은 평균 수명을 높인 장수...","{'사실': 0.4, '미술': 0.27, '화가': 0.2, '사랑': 0.2, ..."
3,S000211518386,홀리데이 인 뮤지엄,"[홀리데이, 뮤지엄]","[홀리데이, 인, 뮤, 지엄]","[홀, 홀리데이, 리, 데이, 뮤]","[홀리데이 인, 뮤지, 엄]",19000,17100,한이준,흐름출판,2023-12-04,예술/대중문화,"★\n10년간 70개 이상, 3,000회 이상의 해설을 진행한 도슨트에게 ‘직접’ ...","{'화가': 0.35, '미술관': 0.23, '소개': 0.23, '박수근': 0...","추천의 글\n프롤로그_ 내 작은 가방 속, 최소한의 미술 이야기\n\n1부. 국내 ...","{'둘러보기': 0.35, '미술관': 0.27, '화가': 0.25, '그림': ..."
4,S000211613105,포 어스 For Earth For Us,"[포, 어스]","[포, 어스]","[포, 어스]","[포, 어스]",35000,31500,케이채,웨일북(whalebooks),2023-12-11,예술/대중문화,"“내가 사랑하는 모든 색은 지구에게 배웠습니다”\n\n총 2,500일, 2만 킬로미...","{'지구': 0.35, '사진': 0.28, '우리': 0.21, '대자연': 0....",프롤로그_자연이야말로 가장 위대한 화가입니다\nMap of Color Travel\...,"{'남극': 0.22, '제도': 0.2, '국립': 0.2, '공원': 0.2, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2884,S000211513171,시네 아프리카,"[시, 아프리카]","[시, 아프리카]","[시, 아프리카]","[시네, 아프리카]",26000,23400,이은별,한울,2023-02-13,예술/대중문화,우리가 생각하는 ‘그’ 아프리카는\n과연 실재하는 아프리카일까?\n영화 〈부시맨〉에...,"{'아프리카': 0.55, '시선': 0.17, '저자': 0.15, '미디어': ...",추천사: 아프리카의 미래 정체성에 대한 소고 _ 마동훈\n프롤로그: 왜 아프리카 영...,"{'아프리카': 0.46, '미래': 0.46, '우리': 0.1, '목소리': 0..."
3112,S000209650972,세계사와 함께 알아보는 음악 이야기,"[세계사, 음악, 이야기]","[세계사, 음악, 이야기]","[세계사, 음악, 이야기]","[세계사, 음악, 이야기]",30000,27000,정봉교,메이킹북스,2023-09-15,예술/대중문화,본서는 고대 그리스 신화 속에서 시작된 ‘Music(음악)’의 어원과 신들에게 제사...,"{'시대': 0.57, '음악': 0.32, '그리스': 0.23, '자연': 0....","1. 고대 그리스 로마 시대(Greece, Roma)\nⅠ. 고대 그리스, 로마 시...","{'시대': 0.6, '음악': 0.32, '주의': 0.27, '배경': 0.27..."
9843,S000211648724,사진가를 위한 캡쳐원 가이드북,"[사진가, 캡쳐원, 가이드북]","[사진가, 위, 캡쳐, 가이드북]","[사진가, 캡, 원, 가이드북]","[사진가, 캡, 원, 가이드북]",30000,27000,박무웅 외,영진닷컴,2023-12-22,예술/대중문화,"사진 촬영, 편집, 관리를 위한 캡쳐원의 모든 것!\n사진가를 위한 캡쳐원 가이드북...","{'캡쳐': 0.38, '기능': 0.25, '사진': 0.23, '촬영': 0.2...","[기본편] 설치, 시작(가져오기 & 테더링 촬영), 보정, 내보내기\nPART 01...","{'방법': 0.43, '캡쳐': 0.38, '사진': 0.25, '설정': 0.1..."
10119,S000208958878,인왕ㆍ인왕 2: 공식 아트웍스,"[인왕, 인왕, 공식, 아트웍스]","[인왕, 인왕, 공식, 아트웍, 스]","[인왕, 공식, 아트, 아트웍, 웍]","[인, 왕, 인, 왕, 공식]",39000,35100,전격게임 편집부,비엘북스,2023-09-25,예술/대중문화,"팀닌자에서 개발하고 코에이테크모, 소니인터렉티브 엔터테인먼트에서 배급서비스 중인 [...","{'인왕': 0.37, '무기': 0.21, '캐릭터': 0.2, '요괴': 0.2...",1. 무대\n- 제단\n- 사체 결계\n- 결계\n- 대장간\n- 인왕\n: 잉글랜...,"{'야마시로국': 0.24, '미노국': 0.19, '셋쓰국': 0.17, '마사'..."


In [422]:
li['유사도 점수'].sort_values()[-5:]

5578    0.217139
4799    0.219679
8198    0.221714
9602    0.230028
6057    0.232875
Name: 유사도 점수, dtype: float64

In [423]:
test

{'인류학': 0.36, '자본주의': 0.33, '영향': 0.2, '인간': 0.2, '사회': 0.2}

In [430]:
li.loc[8198]['소개글 키워드']

{'인간': 0.34, '생각 우리': 0.3, '당신 진화': 0.27}

In [429]:
li.loc[8198]['상품명']

'이토록 뜻밖의 뇌과학'

# SandBox

In [432]:
preprocess('안녕')

[]

In [193]:
input_key = test
data_key = li['소개글 키워드'][0]
score = 0
in_keys = set(input_key.keys())
da_keys = set(data_key.keys())
same = in_keys & da_keys
for i in list(same) :
    score += input_key[i] + data_key[i]
print(score)

0.9


In [191]:
input_key['에디터']

0.23

In [155]:
def DicTrans(keyword) :

    intro = [i[0] for i in keyword]

    dic = {}
    for ind, i in enumerate(intro) :
        dic[i] = keyword[ind][1]

    return dic

In [223]:
li['소개글 키워드'] = li['소개글 키워드'].progress_apply(lambda a : DicTrans(a))

10165it [05:22, 31.49it/s]


In [224]:
li['목차 키워드'] = li['목차 키워드'].progress_apply(lambda a : DicTrans(a))

100%|██████████| 10165/10165 [00:00<00:00, 94815.04it/s]


In [187]:
li['소개글 키워드'][0]

{'에디터': 0.23, '창조': 0.22, '시대': 0.21, '기획': 0.21, '워드': 0.19}

In [318]:
test = {'에디터': 0.23, '창조': 0.22, '영향': 0.2, '인간': 0.2, '사회': 0.2}

In [344]:
test

{'인류학': 0.36, '자본주의': 0.33, '영향': 0.2, '인간': 0.2, '사회': 0.2}

In [343]:
test = DicTrans(string)

In [342]:
string = preprocess(string)

In [153]:
string = li.loc[0]['목차 키워드']
string

[['재료', 0.32], ['거리', 0.27], ['가능성', 0.22], ['리얼', 0.15], ['요점', 0.15]]

In [24]:
'인류학'.split()

['인류학']

In [61]:
런아ㅣ런ㅁ이
if lenfjsd == 0 :
    {'단어' : 1}

[]

In [341]:
string = '이 과제는 인류학적 접근을 통해 자본주의 경제 시스템이 인간 사회와 사회와 문화에 미치는 다양하고 복잡한 영향을 체계적으로 탐구하며, 학생들은 자본주의의 형성과 확산, 소비문화, 불평등, 글로벌화, 환경파괴 등 다양한 측면에서 인류학의 독특한 관점을 활용하여 현대 사회의 복잡한 사회 경제적 문제와 그 영향을 사회학적, 문화인류학적, 공동체적 시각에서 심층적으로 이해하고 분석하며, 인간의 역사와 문화와 함께 성장한 자본주의와의 관계에 대한 통찰력을 개발하고자 하는 목적을 달성하기 위한 레포트를 작성하도록 합니다.'

In [ ]:
with open('Book_List_12_17.pkl', 'rb') as f :
    li = pickle.load(f)

In [431]:
with open('intro.pkl', 'rb') as f :
    intro = pickle.load(f)
intro

0        [[에디터, 0.23], [창조, 0.22], [시대, 0.21], [기획, 0.2...
1        [[루빈, 0.37], [우리, 0.21], [예술가, 0.21], [창조, 0.1...
2        [[화가, 0.44], [미술, 0.35], [구석, 0.35], [영혼, 0.12...
3        [[화가, 0.35], [미술관, 0.23], [소개, 0.23], [박수근, 0....
4        [[지구, 0.35], [사진, 0.28], [우리, 0.21], [대자연, 0.2...
                               ...                        
10391    [[언어, 0.48], [변수, 0.32], [연산자, 0.32], [조건문, 0....
10392    [[로봇, 0.44], [학자, 0.36], [진짜, 0.36], [현장, 0.27...
10393    [[로봇, 0.45], [과학, 0.45], [대회, 0.14], [마음, 0.13...
10394    [[건설, 0.32], [기술, 0.32], [인공, 0.29], [지능, 0.29...
10395    [[로봇, 0.44], [과학, 0.44], [코딩, 0.14], [대한, 0.13...
Name: 소개글 키워드, Length: 10165, dtype: object